# Mini proyecto kaggle

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers

In [2]:
tf.config.list_physical_devices('GPU')


[]

In [3]:
hist = []

In [4]:
train = pd.read_csv('./data/train.csv')
evaluetion = pd.read_csv('./data/test.csv')



In [5]:
train.head(1)

,ID,Y,V1,V2,V3,V4,V5,V6,V7,V8,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
0,1,2013,44.81144,0.83826,0,7.91314,10.94148,-0.04547,-15.16332,-10.47324,...,-8.44873,-230.30484,-40.94698,48.20025,-0.28694,155.76251,-56.23579,13.62599,123.92018,10.02845


## 1. Explorar data

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77779 entries, 0 to 77778
Data columns (total 92 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      77779 non-null  int64  
 1   Y       77779 non-null  int64  
 2   V1      77779 non-null  float64
 3   V2      77779 non-null  float64
 4   V3      77779 non-null  int64  
 5   V4      77779 non-null  float64
 6   V5      77779 non-null  float64
 7   V6      77779 non-null  float64
 8   V7      77779 non-null  float64
 9   V8      77779 non-null  float64
 10  V9      77779 non-null  float64
 11  V10     77779 non-null  float64
 12  V11     77779 non-null  float64
 13  V12     77779 non-null  float64
 14  V13     77779 non-null  float64
 15  V14     77779 non-null  float64
 16  V15     77779 non-null  float64
 17  V16     77779 non-null  float64
 18  V17     77779 non-null  float64
 19  V18     77779 non-null  float64
 20  V19     77779 non-null  float64
 21  V20     77779 non-null  float64
 22

Se cuenta cuantas clases hay por columna

In [7]:
len(train['Y'].value_counts())

74

In [8]:
for c in train.columns:
    print(f'{c} = {len(train[c].value_counts())}')

ID = 77779
Y = 74
V1 = 76309
V2 = 75683
V3 = 1
V4 = 77216
V5 = 77119
V6 = 76607
V7 = 77386
V8 = 77034
V9 = 76852
V10 = 76373
V11 = 77483
V12 = 77584
V13 = 77278
V14 = 77750
V15 = 77752
V16 = 77753
V17 = 77736
V18 = 77738
V19 = 77725
V20 = 77716
V21 = 77701
V22 = 77692
V23 = 77692
V24 = 77665
V25 = 77633
V26 = 77740
V27 = 77738
V28 = 77695
V29 = 77673
V30 = 77687
V31 = 77645
V32 = 77616
V33 = 77595
V34 = 77503
V35 = 77461
V36 = 77643
V37 = 77736
V38 = 77737
V39 = 77715
V40 = 77700
V41 = 77663
V42 = 77656
V43 = 77585
V44 = 77456
V45 = 77504
V46 = 77551
V47 = 77732
V48 = 77712
V49 = 77690
V50 = 77652
V51 = 77649
V52 = 77602
V53 = 77584
V54 = 77633
V55 = 77524
V56 = 77711
V57 = 77715
V58 = 77708
V59 = 77702
V60 = 77675
V61 = 77559
V62 = 77554
V63 = 77431
V64 = 77707
V65 = 77697
V66 = 77666
V67 = 77643
V68 = 77629
V69 = 77638
V70 = 77446
V71 = 77712
V72 = 77712
V73 = 77681
V74 = 77608
V75 = 77595
V76 = 77259
V77 = 77711
V78 = 77692
V79 = 77682
V80 = 77672
V81 = 77364
V82 = 77706
V83 = 77669

> V3 podrias ser eliminado ????????????

## 2. Implementar NN

### 2.1 Preparación 

In [9]:
data = train.drop(columns=['ID', 'V3'])

X = data.drop(columns=['Y']).values
y = data['Y'].values

# División de los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Escala datos de validavión
data_eval= evaluetion.drop(columns=['ID', 'V3'])
data_eval = scaler.transform(data_eval)

c:\Users\facio\anaconda3\envs\tfproject\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [10]:
data_eval= evaluetion.drop(columns=['ID', 'V3'])

### 2.2 Construcción de modelo

### Descripción


El modelo de red neuronal Keras para construir una arquitectura de red más amplia para la regresión. A continuación, se describe los componentes del modelo y cómo se integra con un proceso de validación cruzada para evaluar su rendimiento:

#### 1. Estructura del Modelo:
**Capas Densas:***
Primera capa: 128 neuronas, función de activación ReLU, entrada ajustada al número de características de X.
Segunda capa: 64 neuronas, función de activación ReLU.
Tercera capa: 32 neuronas, función de activación ReLU.
Capa de salida: 1 neurona, función de activación lineal para la regresión.

**Regularización:**
BatchNormalization: Se aplica después de cada capa densa antes del dropout para normalizar las activaciones de la capa anterior.
Dropout: Se utiliza para reducir el sobreajuste al "desconectar" aleatoriamente un porcentaje de neuronas durante el entrenamiento. Los porcentajes son 0.2 después de la primera capa densa y 0.1 después de la segunda.

**Compilación del Modelo:**
Optimizador: Adam con una tasa de aprendizaje de 0.001.
Función de pérdida: Mean Squared Error (MSE), apropiada para problemas de regresión.
Métricas: Mean Absolute Error (MAE), proporciona una medida de la precisión del modelo en términos de error absoluto medio.


#### 2. Entrenamiento y Evaluación:
**Pipeline de Preprocesamiento:**
StandardScaler: Normaliza las características de entrada para tener media 0 y varianza 1, lo que a menudo mejora el rendimiento del entrenamiento en modelos de aprendizaje profundo.
KerasRegressor:
Envoltura que permite utilizar un modelo de Keras como un estimador en scikit-learn.
Configurado para entrenar durante 100 épocas con un tamaño de lote de 256.

**Validación Cruzada:**
KFold: Se usa para dividir los datos en 10 subconjuntos ("folds"), cada uno de los cuales se usa una vez como conjunto de prueba mientras que los otros 9 forman el conjunto de entrenamiento.

**Shuffle:** Los datos se mezclan antes de dividirlos para garantizar que los datos de prueba y entrenamiento sean aleatorios.
Random_state: Fijado para garantizar la reproducibilidad de la división de datos.

**Evaluación:**
Se utiliza cross_val_score para evaluar el pipeline en los diferentes folds con la métrica de neg_mean_squared_error. Esto proporciona una estimación robusta del error cuadrático medio del modelo en diferentes subconjuntos del dataset.

In [23]:
def wider_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(32, activation='relu',  kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        # Dropout(0.1),
        Dense(1, activation='linear') 
    ])     

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    return model

In [29]:
seed = 21
np.random.seed(seed)
estimators = []

estimators.append(('standardize', StandardScaler()))
# Callback de parada temprana
early_stopping = EarlyStopping(monitor='loss', patience=500, verbose=0)

# Añadir el estimador con el callback de parada temprana
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=256, verbose=2, callbacks=[early_stopping])))

pipeline = Pipeline(estimators)


C:\Users\facio\AppData\Local\Temp\ipykernel_61092\3413852010.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=256, verbose=2, callbacks=[early_stopping])))


In [31]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_mean_squared_error', verbose=0)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
274/274 - 2s - loss: 3998756.7500 - mae: 1999.6654 - 2s/epoch - 8ms/step
Epoch 2/100
274/274 - 1s - loss: 3946157.2500 - mae: 1986.4661 - 937ms/epoch - 3ms/step
Epoch 3/100
274/274 - 1s - loss: 3847623.2500 - mae: 1961.4966 - 911ms/epoch - 3ms/step
Epoch 4/100
274/274 - 1s - loss: 3709196.2500 - mae: 1925.8722 - 868ms/epoch - 3ms/step
Epoch 5/100
274/274 - 1s - loss: 3537687.2500 - mae: 1880.7998 - 878ms/epoch - 3ms/step
Epoch 6/100
274/274 - 1s - loss: 3339662.7500 - mae: 1827.3796 - 875ms/epoch - 3ms/step
Epoch 7/100
274/274 - 1s - loss: 3121249.0000 - mae: 1766.5873 - 874ms/epoch - 3ms/step
Epoch 8/100
274/274 - 1s - loss: 2888066.2500 - mae: 1699.2898 - 872ms/epoch - 3ms/step
Epoch 9/100
274/274 - 1s - loss: 2645248.5000 - mae: 1626.2533 - 885ms/epoch - 3ms/step
Epoch 10/100
274/274 - 1s - loss: 2397469.5000 - mae: 1548.1791 - 870ms/epoch - 3ms/step
Epoch 11/100
274/274 - 1s - loss: 2148970.7500 - mae: 1465.7103 - 1s/epoch - 4ms/step
Epoch 12/100
274/274 - 1s - loss: 19

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://2ddc9cf7-420a-4015-9372-71227eb97b2d/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [20]:
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -73.06 (2.58) MSE


In [21]:
pipeline.fit(X,y)

Epoch 1/150
304/304 - 2s - loss: 3996422.5000 - mae: 1999.0809 - 2s/epoch - 8ms/step
Epoch 2/150
304/304 - 1s - loss: 3932087.0000 - mae: 1982.9177 - 992ms/epoch - 3ms/step
Epoch 3/150
304/304 - 1s - loss: 3812498.0000 - mae: 1952.5122 - 995ms/epoch - 3ms/step
Epoch 4/150
304/304 - 1s - loss: 3646120.5000 - mae: 1909.4089 - 998ms/epoch - 3ms/step
Epoch 5/150
304/304 - 1s - loss: 3442168.2500 - mae: 1855.2083 - 1s/epoch - 3ms/step
Epoch 6/150
304/304 - 1s - loss: 3209349.0000 - mae: 1791.3350 - 1s/epoch - 3ms/step
Epoch 7/150
304/304 - 1s - loss: 2955662.0000 - mae: 1719.0417 - 1s/epoch - 3ms/step
Epoch 8/150
304/304 - 1s - loss: 2688345.2500 - mae: 1639.4200 - 993ms/epoch - 3ms/step
Epoch 9/150
304/304 - 1s - loss: 2413925.5000 - mae: 1553.4474 - 983ms/epoch - 3ms/step
Epoch 10/150
304/304 - 1s - loss: 2138271.0000 - mae: 1462.0078 - 985ms/epoch - 3ms/step
Epoch 11/150
304/304 - 1s - loss: 1866649.2500 - mae: 1365.9329 - 988ms/epoch - 3ms/step
Epoch 12/150
304/304 - 1s - loss: 1603755.

Pipeline(steps=[('standardize', StandardScaler()),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x000001940D88E410>)])

In [ ]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, validation_split=0.3, epochs=150, batch_size=256, verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

In [ ]:
# Evaluación del modelo en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

487/487 [==============================] - 1s 1ms/step - loss: 1223.3323 - mae: 26.9070
Test Loss: 1223.332275390625, Test MAE: 26.90700340270996


# Predicción

In [22]:
def save_data(model, name):
    y_predict = model.predict(data_eval)

    df = evaluetion[['ID']]
    df['Y'] = y_predict
    df.to_csv(f'predict/{name}.csv', index=False)

save_data(pipeline, 'crossvalidation_regularizacion')

c:\Users\facio\anaconda3\envs\tfproject\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


73/73 - 0s - 213ms/epoch - 3ms/step


C:\Users\facio\AppData\Local\Temp\ipykernel_61092\674811728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Y'] = y_predict


In [ ]:
import pickle

def save_model(model, name):
    with open(f'./models/{name}.pkl', 'wb') as archivo_salida:
        pickle.dump(model, archivo_salida)

save_model(model, 'dropout-sample')

INFO:tensorflow:Assets written to: ram://ce156ec7-9e37-46b8-bda6-703c9aaca2d2/assets


In [ ]:
hist.append({
    'desc': 'normalizada con 2 podas aumentando porcentaje de poda',
    'mae':  5.834270477294922,
    'mae_kaggle': None # pendiente
})

pd.DataFrame(hist)

,n,desc,mae,mae_kaggle
0,5,normalizada con una poda,6.186801,9.16731
1,NaN,normalizada con 2 podas,5.971045,NaN
2,NaN,normalizada con 2 podas aumentando porcentaje ...,5.843672,NaN
3,NaN,normalizada con 2 podas aumentando porcentaje ...,5.834270,NaN
